### Modelling sequence learning

As in Whittington et al. (2020), we model the spatial task of predicting the next location in a trajectory as the prediction of the next node in a graph. We create a large set of graphs, each one an n-by-n grid of nodes representing a simple spatial environment. Nodes are labelled with random letters to represent arbitrary associations at a particular location. Each directed edge, i.e. each possible transition in the graph, is of the type north, south, east, or west. Random walks in the set of graphs are used to train the model; these could represent sequences stored in an initial bank of memories. The generative model is trained from scratch on the replayed sequences (converted to strings of the form ‘node1 E node2 W node3 …’) with the mechanism of causal language modelling.

Tested with conda_pytorch_latest_p36 kernel in AWS SageMaker.

#### Installation:

In [ ]:
!pip install simpletransformers csrgraph networkx

#### Imports:

In [ ]:
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import csrgraph as cg
import numpy as np
import random
import string
from simpletransformers.language_modeling import (
    LanguageModelingModel,
    LanguageModelingArgs,
)
from simpletransformers.language_generation import (
    LanguageGenerationModel, 
    LanguageGenerationArgs,
)

#### Prepare training data

In [ ]:
def get_graph(nodes = ["a", "b", "c", "d", "e", "f", "g", "h", "i"]):

    G = nx.DiGraph()
    east_pairs = [(nodes[0], nodes[1]), (nodes[1], nodes[2]), (nodes[3], nodes[4]), 
                  (nodes[4], nodes[5]), (nodes[6], nodes[7]), (nodes[7], nodes[8])]
    south_pairs = [(nodes[0], nodes[3]), (nodes[3], nodes[6]), (nodes[1], nodes[4]), 
                   (nodes[4], nodes[7]), (nodes[2], nodes[5]), (nodes[5], nodes[8])]
    north_pairs = [(i[1], i[0]) for i in south_pairs]
    west_pairs = [(i[1], i[0]) for i in east_pairs]

    for n in nodes:
        G.add_node(n)

    for tple in east_pairs:
        G.add_edge(tple[0], tple[1], direction='E')
    for tple in north_pairs:
        G.add_edge(tple[0], tple[1], direction='N')
    for tple in west_pairs:
        G.add_edge(tple[0], tple[1], direction='W')
    for tple in south_pairs:
        G.add_edge(tple[0], tple[1], direction='S')

    return G

In [ ]:
def plot_G(G):
    pos = nx.spring_layout(G, iterations=100, seed=39775)

    fig, ax = plt.subplots(1)

    nx.draw(G, pos, ax=ax, font_size=8, with_labels=True)

    fig.tight_layout()
    plt.show()

#plot_G(G)

In [ ]:
def get_random_walks(G):
    csr_G = cg.csrgraph(G, threads=12) 
    node_names = csr_G.names
    walks = csr_G.random_walks(walklen=50, # length of the walks
                    epochs=10, 
                    start_nodes=None, 
                    return_weight=1.,
                    neighbor_weight=1.)

    walks = np.vectorize(lambda x: node_names[x])(walks)
    return walks

In [ ]:
def walk_to_string(walk, G):
    walk_string = ""
    for i in range(len(walk)-1):
        node1 = walk[i]
        node2 = walk[i+1]
        direc = G.edges[(node1, node2)]['direction']
        walk_string += str(node1) + " "+ str(direc) + " "
    walk_string += walk[-1]
    return walk_string

def get_walks_as_strings():
    entities_for_graphs =[[random.choice(string.ascii_letters[0:26]) for i in range(9)] for i in range(1000)]
    entities_for_graphs = [entities for entities in entities_for_graphs if len(list(set(entities)))== 9]

    walks_as_strings = []
    for nodes in entities_for_graphs:
        G = get_graph(nodes=nodes)
        walks = get_random_walks(G)
        walks_as_strings.extend([walk_to_string(walk, G) for walk in walks])
    return walks_as_strings

walks_as_strings = get_walks_as_strings()

#### Train generative model

Train GPT-2 from scratch on dataset created above.

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = LanguageModelingArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 10
model_args.dataset_type = "simple"
model_args.save_model_every_epoch = False
model_args.evaluate_during_training = True
model_args.mlm = False  # mlm must be False for CLM
model_args.learning_rate = 1e-5
model_args.vocab_size=100
model_args.use_early_stopping = True
model_args.manual_seed = 123

text_file = open("train.txt", "w")
walks = get_walks_as_strings()[0:10000]
shuffle(walks)
n = text_file.write('\n'.join(walks))
text_file.close()

text_file = open("test.txt", "w")
walks = get_walks_as_strings()[0:1000]
shuffle(walks)
n = text_file.write('\n'.join(walks))
text_file.close()

train_file = "train.txt"
test_file = "test.txt"

model = LanguageModelingModel(
    "gpt2", None, train_files='train.txt', args=model_args
)

# Train the model
model.train_model(train_file, eval_file=test_file)

# Evaluate the model
result = model.eval_model(test_file)

#### Load trained model for sequence generation

In [ ]:
model = LanguageGenerationModel("gpt2", "outputs", args={'do_sample': False, 'evaluate_generated_text': True})

In [ ]:
model.generate("a E b S e W d N")

In [ ]:
loops = ["{} E {} S {} W {} N", "{} S {} W {} N {} E", "{} W {} N {} E {} S", "{} N {} E {} S {} W",
        "{} E {} N {} W {} S", "{} N {} W {} S {} E", "{} W {} S {} E {} N", "{} S {} E {} N {} W"]

In [ ]:
def test_loop():
    random_nodes = [random.choice(string.ascii_letters[0:26]) for i in range(4)]
    loop = random.choice(loops)
    test_string = loop.format(random_nodes[0], random_nodes[1], random_nodes[2], random_nodes[3])
    output = model.generate(test_string)
    output = output[0][0:len(test_string)+2]
    if output[-1] == output[0]:
        return 1
    else:
        return 0

results = [test_loop() for i in range(100)]

In what percentage of trials was the next node correct?

In [ ]:
results.count(1)

#### A more challenging test

For an arbitrary loop in the graph, can the model predict the final item?

In [ ]:
def get_cycles_for_graph(G):
    cycles = nx.simple_cycles(G)
    loops = []
    for c in cycles:
        path_string = ""
        for ind, node in enumerate(c):
            if ind+1 < len(c):
                direction = G.get_edge_data(c[ind], c[ind+1])['direction']
                path_string += '{} {} '.format(node, direction)
            else:
                direction = G.get_edge_data(c[ind], c[0])['direction']
                path_string += '{} {} '.format(node, direction)
        loops.append(path_string)
    return loops

def test_loop(num_graphs = 5):
    results = []
    lens = []
    
    for i in range(num_graphs):
        entities_for_graphs =[[random.choice(string.ascii_letters[0:26]) for i in range(9)] for i in range(100)]
        entities_for_graphs = [entities for entities in entities_for_graphs if len(list(set(entities)))== 9]
        nodes = entities_for_graphs[0]
        G = get_graph(nodes=nodes)
        test_strings = get_cycles_for_graph(G)

        for test_string in test_strings:
            lens.append((len(test_string))/4)
            output = model.generate(test_string)
            output = output[0][0:len(test_string)+1]
            if output[-1] == output[0]:
                results.append(1)
            else:
                results.append(0)
    
    return results, lens

results, lens = test_loop()

#### Plot structural inference accuracy against graph cycle length

In [ ]:
def acc_for_len(length):
    accs = [r for ind, r in enumerate(results) if lens[ind] == length]
    return accs.count(1) / len(accs)

lengths = [2, 4, 6, 8]
accuracies = [acc_for_len(i) for i in lengths]

plt.bar(lengths, accuracies, width=1.3, color='skyblue')
plt.title('Next node inference accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of transitions')
plt.tight_layout()
plt.savefig('graph_cycle_length.png')

In [ ]:
plt.figure()
plt.rcParams.update({'font.size' : 15})

df = pd.read_csv('outputs/training_progress_scores.csv')
df = df.iloc[0:6]
df.plot(x='global_step', y='eval_loss', title='Loss over time', 
                   ylabel='Loss on test set', xlabel = 'Training step', legend=False)

plt.tight_layout()
plt.savefig('graph-gpt.png')

#### Test ability to generate valid graphs

In [ ]:
model = LanguageGenerationModel("gpt2", "outputs", args={'do_sample': True, 'max_length': 50})

In [ ]:
model.generate("ia ")

In [ ]:
with open('train.txt', 'r') as fh:
    lines = fh.readlines()
    
all_train_txt = ' '.join(lines)

def test_sequences(seq):
    N_num = 0
    S_num = 0
    E_num = 0
    W_num = 0
    for ind, char in enumerate(seq):
        if char in ['N', 'S', 'E', 'W']:
            if char == 'N':
                N_num += 1
            if char == 'S':
                S_num += 1
            if char == 'E':
                E_num += 1
            if char == 'W':
                W_num += 1
            if N_num == S_num and E_num == W_num:
                print("Found loop.")
                print(seq[0:ind+3])
                if seq[0:ind] in all_train_txt:
                    print("In training, skipping loop.")
                else:
                    if seq[0] == seq[ind+2]:
                        print(seq[0], seq[ind+2])
                        return 1, ind
                    else: 
                        return 0, ind
    return None, ind
                
def generate_seq():  
    rand_letter = random.choice(string.ascii_letters[0:26])
    return model.generate(rand_letter)[0]

results = [test_sequences(generate_seq()) for i in range(1000)]
correct = [i[0] for i in results]
lengths = [i[1] for i in results]

In [ ]:
def acc_for_len(length):
    accs = [r for ind, r in enumerate(correct) if lengths[ind] == length]
    return accs.count(1) / len(accs)

lengths_set = [6, 14, 22, 30]
accuracies = [acc_for_len(i) for i in lengths_set]
lengths_set = [2, 4, 6, 8]

plt.figure()
plt.rcParams.update({'font.size' : 15})

plt.bar(lengths_set, accuracies, width=1.3, color='skyblue')
plt.title('Correctness of generated graphs')
plt.ylabel('Fraction correct')
plt.xlabel('Number of transitions')
plt.xticks([2, 4, 6, 8])
plt.tight_layout()
plt.savefig('graph_cycle_length_gen.png')